In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
os.listdir('/content/gdrive/My Drive/vc_modeling')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


['data']

In [4]:
file_names = os.listdir("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/")
print(file_names)

['category_groups.csv', 'funding_rounds.csv', 'people.csv', 'checksum.csv', 'people_descriptions.csv', 'investors.csv', 'organization_descriptions.csv', 'investment_partners.csv', 'event_appearances.csv', 'organizations.csv', 'org_parents.csv', 'jobs.csv', 'acquisitions.csv', 'funds.csv', 'ipos.csv', 'degrees.csv', 'investments.csv', 'events.csv']


In [5]:
df_names = [x[:-4] for x in file_names]
print(df_names)

['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events']


In [6]:
dfs = [pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/"+x) for x in file_names]
df_dict = dict(zip(df_names, dfs))
print(df_dict.keys())

dict_keys(['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events'])


In [8]:
funding_df = df_dict["funding_rounds"]
funding_df.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,143236.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,500000.0,USD,NaN,NaN,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,145939.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,273250.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,California,Menlo Park,series_b,2006-04-01,27500000.0,27500000.0,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,photobucket-series-b--bcd5a63d,https://www.crunchbase.com/funding_round/photo...,149479.0,2007-05-29 11:05:59,2018-02-12 23:27:36,USA,CO,Colorado,Denver,series_b,2006-05-01,10500000.0,10500000.0,USD,NaN,NaN,USD,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,geni-series-a--60e6afd9,https://www.crunchbase.com/funding_round/geni-...,313010.0,2007-05-31 20:19:28,2018-02-12 23:41:29,USA,CA,California,West Hollywood,series_a,2007-01-17,NaN,NaN,NaN,10000000.0,10000000.0,USD,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c


In [11]:
num_funding_rounds = funding_df.groupby("org_uuid", as_index="False").size().reset_index(name="num_funding_rounds")
num_funding_rounds.head()

,org_uuid,num_funding_rounds
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,1
1,000014da-0c46-b9cb-0941-3a93c027b119,7
2,00002470-bff7-6226-5800-0ca1b3787b6f,5
3,000095de-8e2b-82f1-32a7-c222ba3d5682,1
4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,2


In [21]:
avg_funding_amt_usd = funding_df.groupby("org_uuid", as_index="False").mean(numeric_only=True)[["raised_amount_usd"]].dropna(how="all", subset=["raised_amount_usd"])
avg_funding_amt_usd.head()

,raised_amount_usd
org_uuid,
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,8.260736e+07
00002470-bff7-6226-5800-0ca1b3787b6f,1.133333e+06
000095de-8e2b-82f1-32a7-c222ba3d5682,9.500000e+06
0000d497-c93a-eea3-eeb0-a943dfb4f71e,4.315760e+05
0001a8cc-0cdc-4a30-b4d3-da1b425069e1,1.500000e+06


In [25]:
funding_features = num_funding_rounds.join(avg_funding_amt_usd, on="org_uuid")
funding_features[:100]

,org_uuid,num_funding_rounds,raised_amount_usd
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,1,8.260736e+07
1,000014da-0c46-b9cb-0941-3a93c027b119,7,NaN
2,00002470-bff7-6226-5800-0ca1b3787b6f,5,1.133333e+06
3,000095de-8e2b-82f1-32a7-c222ba3d5682,1,9.500000e+06
4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,2,4.315760e+05
...,...,...,...
95,00295b25-f235-462c-a096-90eee885b296,1,NaN
96,002aac96-79d4-a8f3-8ef0-d44fb240951d,1,3.000000e+06
97,002ad549-e548-4dcd-9eb6-1029c7590c4b,1,2.236403e+06
98,002b904e-7583-3ab0-3151-d130e0db7561,1,1.000000e+03


In [28]:
from google.colab import files

# export features to csv
funding_features.to_csv('funding_features.csv', columns = ["org_uuid", "num_funding_rounds", "raised_amount_usd"], index=False) 
files.download('funding_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>